# FISP Twitter Projects Notebook

This notebook contains the latest code for pulling tweets and cleaning, manipilating the dataset.

In [7]:
# coding: utf-8

In [8]:
# import necessary python packages
import sys
#sys.path.append("/usr/local/lib/python2.7/site-packages")
import tweepy #https://github.com/tweepy/tweepy
# import dropbox #https://www.dropbox.com/developers-v1/core/docs/python
import csv
import time
import os
from datetime import datetime
from collections import defaultdict
import logging
import gspread
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from unidecode import unidecode
import xlsxwriter

#Twitter and Dropbox API credentials
import api_cred as ac

In [9]:
# setup debug logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [10]:
# modify print precison for easier debugging
np.set_printoptions(precision=20)

# Helper Functions

In [11]:
def authenticate_twitter():
  auth = tweepy.OAuthHandler(ac.consumer_key, ac.consumer_secret)
  auth.set_access_token(ac.access_key, ac.access_secret)
  api = tweepy.API(auth)
  return api

In [12]:
def get_new_tweets(tweet_name, since_id):
  api = authenticate_twitter()
  tweets = []
  new_tweets = api.user_timeline(screen_name = tweet_name, since_id = since_id, count = 200)
  tweets.extend(new_tweets)
  if len(tweets) > 0:
    max_id = tweets[-1].id - 1
  while (len(new_tweets) > 0):
    new_tweets = api.user_timeline(screen_name = tweet_name, since_id = since_id, count = 200, max_id = max_id)
    tweets.extend(new_tweets)
    max_id = tweets[-1].id - 1
  
  tweets = [[tweet.id_str, tweet.created_at, tweet.text, "", "", "",tweet.retweet_count, tweet.favorite_count] for tweet in tweets]
  logger.info("Downloading %d tweets from %s" % (len(tweets), tweet_name))
  return tweets[::-1]

In [13]:
def get_lists(df):
  # put twitter handles, last acquired tweet ID, tweet count and store them in respective lists
  names = filter(lambda x: x > 0, df.iloc[:, 1])
  max_ids = df.iloc[:, 2]
  counts = df.iloc[:, 3]
  
  # save the number of entries
  indices = range(1,len(names)+1)
  
  lists = zip(names, max_ids, counts, indices)
  del lists[0] # the first one is column title
  return lists

In [14]:
def load_sheets(path):
  sheet_book = load_workbook(path)
  sheet_writer = pd.ExcelWriter(path, engine='openpyxl')
  sheet_writer.book = sheet_book
  sheet_writer.sheets = dict((ws.title, ws) for ws in sheet_book.worksheets)
  logger.info("Downloaded %s" % path)
  return sheet_writer

In [15]:
# needs to be finished
def illchar(par):
  writer = pd.ExcelWriter('id_.xlsx')
  _id = pd.Series(df.iloc[:,0]).astype(str)
  _id.to_excel(writer, sheet_name='id', header=False, index=False, float_format='string')
  writer.save()
  df.to_csv('illchar.csv', encoding='utf-8')

# Write to Sheets ↓

The functions below write data to the currently local sheets.

## Scrape Tweets and save them to respective excel file

*twitter_list.xlsx* contains the list of candidates for each tweets excel file along with the metadata.

*cand_tweets.xlsx* contains the tweets for all announced candidates

*spec_tweets.xlsx* contains the tweets for all the speculated candidates

*rep_tweets.xlsx* contains the tweets for all current CA represenatives of interest for the CA 2018 Elections.

In [16]:
#############################################
# collect_data(tweet_sheet, twitter_list, twitter_sheet)
# This function pulls new latest tweets and appends them to the correct excel file
# params: tweet_sheet - the path to the excel file to save the new tweets 
#         twitter_list - path to file that contains the twitter handles, last tweet pulled id, tweet counts, and 
#                        last pull date
#         twitter_sheet - the correct sheet of accounts corresponding to the tweet sheet passed in
# returns: n/a
def collect_data(tweet_sheet, twitter_list, twitter_sheet):
  # start timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  tweets_path = os.path.expanduser(tweet_sheet)
  twitter_path = os.path.expanduser(twitter_list)
  
  # load and prepare list of twitter accounts    
  list_writer = load_sheets(twitter_path)
  list_df = pd.read_excel(twitter_path, twitter_sheet)
  
  # list_df['Last_Pulled'] = pd.to_datetime(list_df['Last_Pulled'], errors='coerce') # used to fix datetime formatting
  
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(tweets_path)
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for index, row in list_df.iterrows():       
    name, since_id, count = row[1], row[2],row[3]
    
    # check if rep has a twitter handle 
    if (type(name) is not unicode):
      continue
    
    new_tweets = get_new_tweets(name, since_id)
    
    # if there are no new tweets continue to the next account
    if (len(new_tweets) > 0):
      # turn the new tweets into a dataframe and write them to the corresponding excel sheet
      df = pd.DataFrame(new_tweets)
      
      df.to_excel(tweet_writer, sheet_name=name, startrow=count+1, header=False, index=False, float_format='string')
      
      # update since_id, count, and last_pull date in tweet list
      list_df.iat[index,2] = new_tweets[len(new_tweets)-1][0] # since_id
      list_df.iat[index,3] = count + len(new_tweets) # last_pull
      list_df.iat[index,4] = pd.to_datetime(time.strftime("%m/%d/%Y %H:%M:%S"), errors='coerce') # last_pull date
      
      logger.info("Updated new tweets on spreadsheet for %s" % name)
      time.sleep(20)
  
  # write the updated list and save the changes to the excel sheets
  list_df.Last_ID = list_df.Last_ID.astype(str)
  list_df.to_excel(list_writer, sheet_name=twitter_sheet, index=False) #, float_format='string')
  list_writer.save()
  tweet_writer.save()
  
  logger.info("Done appending new tweets for %s", twitter_sheet)
  # stop timer and print time elapsed for the current data pull
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)

In [17]:
# Purpose: Updates like and retweet totals
def collect_addition_data(tweet_sheet, tweet_list, sheetname):
  # start the timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  tweet_sheet = os.path.expanduser(tweet_sheet)
  tweet_list = os.path.expanduser(tweet_list)
  
  # load and prepare list of twitter accounts
  list_writer = load_sheets(tweet_list)
  list_df = pd.read_excel(tweet_list, sheet_name=sheetname)
  list_df = list_df.dropna(thresh=4)
  # properly load spreadsheet to append new data
  tweet_writer = load_sheets(tweet_sheet)
  logger.info("Downloaded tweets list")
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for row in list_df.itertuples():  
    name, since_id, count = row[2], row[3],row[4]
    
    # read cand tweet sheet
    tweets_df = pd.read_excel(tweet_sheet, sheetname=name)
    logger.info("Retrived data from spreadsheet for %s" % name)
    
    # retreive updated tweets
    tweets = get_new_tweets(name, 1)
    updates_df = pd.DataFrame(tweets)
    
    if (updates_df.empty):
      continue 
      
    # clean dataframe to only include id, retweets, and favorites
    updates_df = updates_df[[0, 6, 7]]
    updates_df.columns = ['id', 'retweets', 'favorites']
    
    # call helper fuction to match updated metadata with correct tweets
    tweets_df = update_metadata(tweets_df, updates_df, name)
    
    if (name == 'antonio4ca'):
      continue
      
    # write the updated data to the twitter profile's sheet to be saved
    tweets_df.to_excel(tweet_writer, sheet_name=name, index=False, startcol=1)
    logger.info("Updated data on spreadsheet for %s" % name)
    # 100 second pause between data pulls to avoid token exceptions
    time.sleep(20)
  
  tweet_writer.save()
  
  logger.info("Done collecting additional data")
  # stop timer and print time elapsed for the current data pull
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)

In [18]:
# This function takes the up to date metadata and matches it to their respective tweet using a tweet's unique id
# currently not in use
def update_metadata(tweets_df, updates_df, cand_name): 
  # convert tweet id to the same type as the updates sheet
  tweets_df['id'] = tweets_df['id'].astype(str)
  tweets_df.set_index('id', inplace=True)
  
  ## loop through the updates metadata and updates the tweet sheet
  for row in updates_df.itertuples():
    tweets_df.set_value(row[1], 'retweets', row[2])
    tweets_df.set_value(row[1], 'favorites', row[3])

  # drop null rows that could not match with a tweet
  tweets_df.dropna(subset=['created_at'], inplace=True)
  
  return tweets_df

In [19]:
def create_new_sheets(new_tweet_sheet, new_tweet_list, list_sheet_names):
  # start the timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  new_tweet_sheet = os.path.expanduser(new_tweet_sheet)
  new_tweet_list = os.path.expanduser(new_tweet_list)
  
  # Create a Pandas Excel writer using XlsxWriter as the engine.
  tweet_writer = pd.ExcelWriter(new_tweet_sheet)
  
  # load and prepare list of twitter accounts
  list_writer = load_sheets(new_tweet_list)
  
  # will contain name of every accnt 
  sheet_list = []
  
  for sheetname in list_sheet_names:
    list_df = pd.read_excel(new_tweet_list, sheetname=sheetname)
    list_df = list_df.dropna(axis=0, subset=['Twitter'])
  
    # 
    for row in list_df.itertuples():
      name, since_id, count = row[2], row[3],row[4]
      sheet_list.append(name)
      
      # Create a Pandas dataframe from some data.
      col_headers = {'id': [np.nan], 'created_at': [np.nan], 'text': [np.nan], 'hashtag#': [np.nan], 'at@': [np.nan],
                    'link': [np.nan], 'retweets': [np.nan], 'favorites': [np.nan], 'fullURL': [np.nan]}
      tweets_df = pd.DataFrame(col_headers)
      tweets_df = tweets_df[['id', 'created_at', 'text', 'hashtag#', 'at@', 'link', 'retweets', 'favorites', 'fullURL']]
      
      # write the updated data to the twitter profile's sheet to be saved
      tweets_df.to_excel(tweet_writer, sheet_name=name, index=False, startcol=0)
    
  tweet_writer.save()
  logger.info("Done creating excel doc")
  # stop timer and print time elapsed for the current data pull
  end = time.time()
  logger.info("Time Elapsed: %d", float((end-start))/60)
  return sheet_list

In [25]:
#############################################
# convert_xlsx_csv (tweet_sheet, sheetname, tweet_list)
# This function takes the excel workbook of tweets and converts it all into a single csv of tweets
# params: tweet_sheet - the path to the excel file with the tweets
#         tweet_list - path to file that contains the twitter handles, last tweet pulled id, tweet counts, and 
#                        last pull date
#         sheetname - sheet within in tweet_list
# returns: n/a
def convert_xlsx_csv (tweet_sheet, sheetname, tweet_list):
  # start timer
  start = time.time()
  logger.info("Start...")
  
  # process the paths so they are passable to load_sheets
  tweet_sheet = os.path.expanduser(tweet_sheet)
  tweet_list = os.path.expanduser(tweet_list)  
    
  # load and prepare list of twitter accounts    
  list_writer = load_sheets(tweet_list)
  list_df = pd.read_excel(tweet_list, sheetname=sheetname)
  #merged_corpus = pd.DataFrame(columns=['id', 'created_at', 'text', 'hashtag#', 'at@', 'link', 'retweets', 'favorites', 'full URL'])
  merged_df = pd.DataFrame()

  initial_loop = True
  
  # loop through the list of Cand/PACs and updates each tweet sheet appropriately
  for index, row in list_df.iterrows():
    name = row[0]
    
    if (initial_loop):
      merged_df = pd.read_excel(tweet_sheet, sheetname=name)
      merged_df['Name'] = name
      num_tweets = len(merged_df.index)
      print num_tweets

      logger.info("Retrived data from spreadsheet for %s" % name)
      initial_loop = False
    
    else:
      # read current cand tweet sheet
      curr_df = pd.read_excel(tweet_sheet, sheetname=name)
      curr_df['Name'] = name
      num_tweets = len(curr_df.index)
      if (num_tweets == 0):
        continue
      
      logger.info("Retrived data from spreadsheet for %s" % name)
      
      merged_df = merged_df.append(curr_df)
  
  print merged_df.shape
  # write the updated list and save the changes to the excel sheets
  #merged_df.to_csv('merged_corpus.csv', encoding='utf-8')
  
  logger.info("done")
  return merged_df

In [26]:
# I honestly don't remember what I used this for 
def list_to_number_string(value):
    if isinstance(value, (list, tuple)):
        print str(value) + 'here'
        return str(value)[1:-1]
    else:
        return value

# sample_df['text'] = sample_df['text'].apply(list_to_number_string)

In [22]:
# func that will pull a number of tweets and create a coding sheet with columns for rating
# sentitment, partisanship, policy, fact, and opinion
def coding_sampling(coding_workbook, sheetname, sample_size, exclude):
  logger.info("Start...")
  start = time.time()
  
  # properly load spreadsheet to append new data
  coding_workbook = os.path.expanduser(coding_workbook)
  sample_writer = load_sheets(coding_workbook)
  full_df = pd.read_excel(coding_workbook, sheetname)
  logger.info("Full data loaded")
  
  col_headers = {'id': [str],'text': [np.nan], 'sentiment': [np.nan], 'political': [np.nan], 'ideology': [np.nan],
                 'macroeconomics': [np.nan], 'national_security': [np.nan], 'ideology': [np.nan],
                 'crime_law_enforcement': [np.nan], 'civil_rights': [np.nan], 'environment': [np.nan],
                 'education': [np.nan], 'healthcare': [np.nan], 'governance': [np.nan], 'no_policy_content': [np.nan],
                 'asks_for_donation': [np.nan], 'ask_to_watch_read_share_follow_something': [np.nan],
                 'factual_claim': [np.nan], 'opinion': [np.nan]}
  sample_df = pd.DataFrame(col_headers)
  
  # keep only id and text 
  full_df = full_df[['id', 'text']]
  
  # if (exclude):
    # remove already coded tweets, yet to be implemented
  
  # sample the tweets
  sample_tweets = full_df.sample(sample_size)
  logger.info("Sample made")
  
  # concat the sampled tweets with the coding measurments
  sample_df = pd.concat([sample_tweets, sample_df])
  sample_df = sample_df[['id', 'text', 'sentiment', 'political', 'ideology', 'macroeconomics', 'immigration',
                         'national_security', 'crime_law_enforcement', 'civil_rights', 'environment', 'education',
                         'healthcare', 'governance', 'no_policy_content', 'asks_for_donation',
                         'ask_to_watch_read_share_follow_something', 'factual_claim', 'opinion']]
  sample_df.drop(sample_df.tail(1).index,inplace=True) # drop last n rows
  logger.info("Sample processed and concated")
  
  sample_df.id = sample_df.id.astype(str)
  sample_df.to_excel(sample_writer, 'batch2', index=False, float_format='string')
  sample_writer.save()
  
  logger.info("done")
  #return sample_df

In [ ]:
coding_sampling(state_data_dir + coded_tweets, all_tweets, coding_sample_size, 0)

In [ ]:
# reference code on how to save the data to an excel sheet and not lose id data
writer = pd.ExcelWriter('coding_file.xlsx')
merged_sheets.id = merged_sheets.id.astype(str)
merged_sheets.to_excel(writer, 'total_sample', index=False, float_format='string')
writer.save()

#### Tweets Pull Variables

In [23]:
# set file pathway variables an expand to HOME
ca_data_dir = '~/Dropbox/Summer_of_Tweets/ca_working_sheets/'
state_data_dir = '~/Dropbox/Summer_of_Tweets/State_Leg_Working_Data/'

# the excel sheets containing the tweets
cand_tweets = "cand_tweets.xlsx"
spec_tweets = "spec_tweets.xlsx"
rep_tweets = "rep_tweets.xlsx"
state_tweets = "state_tweets.xlsx"
coded_tweets = "coding_file.xlsx"

# the excel file containing the accounts and its sheets
twitter_list = "Twitter_List.xlsx" # this is the name of the metadata lists
cand_sheet = "cand"
spec_sheet = "speculated"
rep_sheet = "reps"
ca49_sheet = "CA49"
state_sheets = ["CA", "AK", "ME", "IL", "GA", "WY", "MI", "IN","HI", "AL", "CT", "WA", "AZ", "FL", "NJ", "OR", "OK",   
                "SD", "WI", "PA"]
all_handles = 'all'
all_tweets = 'total_sample'

coding_sample_size = 1000

In [49]:
collect_data(ca_data_dir + cand_tweets, ca_data_dir + twitter_list, cand_sheet)
print 
collect_data(ca_data_dir + spec_tweets, ca_data_dir + twitter_list, spec_sheet)
print 
collect_data(ca_data_dir + rep_tweets, ca_data_dir + twitter_list, rep_sheet)
print
collect_data(ca_data_dir + cand_tweets, ca_data_dir + twitter_list, ca49_sheet)

INFO:__main__:Start...
INFO:__main__:Downloaded /Users/SoloMune/Dropbox/Summer_of_Tweets/ca_working_sheets/Twitter_List.xlsx
INFO:__main__:Downloaded /Users/SoloMune/Dropbox/Summer_of_Tweets/ca_working_sheets/cand_tweets.xlsx
INFO:__main__:Downloading 71 tweets from ApplegateCA49
INFO:__main__:Updated new tweets on spreadsheet for ApplegateCA49
INFO:__main__:Downloading 321 tweets from SaraJacobsCA
INFO:__main__:Updated new tweets on spreadsheet for SaraJacobsCA
INFO:__main__:Downloading 1405 tweets from MikeLevinCA
INFO:__main__:Updated new tweets on spreadsheet for MikeLevinCA
INFO:__main__:Downloading 3 tweets from CPforCongress
INFO:__main__:Updated new tweets on spreadsheet for CPforCongress
INFO:__main__:Downloading 124 tweets from KristinDGaspar
INFO:__main__:Updated new tweets on spreadsheet for KristinDGaspar
INFO:__main__:Downloading 58 tweets from DianeHarkey
INFO:__main__:Updated new tweets on spreadsheet for DianeHarkey
INFO:__main__:Downloading 65 tweets from brianlmaryot

In [ ]:
#collect_addition_data(ca_data_dir + cand_tweets, ca_data_dir + twitter_list, cand_sheet)
print
collect_addition_data(ca_data_dir + spec_tweets, ca_data_dir + twitter_list, spec_sheet)
print
collect_addition_data(ca_data_dir + rep_tweets, ca_data_dir + twitter_list, rep_sheet) # <-- issues here
print
collect_addition_data(ca_data_dir + cand_tweets, ca_data_dir + twitter_list, ca49_sheet)

In [ ]:
for state in state_sheets:
  collect_data(state_data_dir + state_tweets, state_data_dir + twitter_list, state)